<a href="https://colab.research.google.com/github/gisazae/Tensorflow-Examples/blob/master/IntegracionCorpus_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SmartSec4COP F. Muñoz, G. Isaza
# Remember the pipeline
# ClasificadorCNN.ipynb	233.87 KB	
Ajustes del clasificador CNN y prubas con el set de datos de PAN12
ClasificadorCNNEmb.ipynb	47.41 KB	
2017‑05‑05
Pruebas antes de GPU, falta un resultado por documentar
ClasificadorCNNTweetToknz.ipynb	52.12 KB	
2017‑03‑07
Iniciando implementacion de perceptro multicapa y haciendo pruebas del clasificador CNN con calculo de embeds incluido.
ClasificadorPerceptron.ipynb	3.18 KB	
2017‑04‑09
Listo para entrenamientos por mensajes
IntegracionCorpus.ipynb	573.68 KB	
2017‑05‑05
Pruebas antes de GPU, falta un resultado por documentar
NPSChats.ipynb	3.4 KB	
2016‑10‑07
Revision del acceso de PAN CLEF y NPS
OTROS Dataset.ipynb	810 B	
2016‑09‑29
Nuevos notebooks creados para descargar y evaluar otros conjuntos de dato, y uno para realizar a integracion.
PAN CLEF corpus.ipynb	455.36 KB	
2017‑03‑18
Ajustando de talles de generacion de matriz de embedds para Integracion de embeds con Lexicon, diccionario Empath (U. Stanford)
PJ data.ipynb	11.29 KB	
2016‑09‑29
Nuevos notebooks creados para descargar y evaluar otros conjuntos de dato, y uno para realizar a integracion.
Results.ipynb	18.47 KB	
2017‑05‑05
Pruebas antes de GPU, falta un resultado por documentar
Test Tensor Flow.ipynb	121.01 KB	
2017‑02‑14
Red CNN para clasificacion en 2 grupos, primeros entrenamientos
TestClasificadorCNNEmb.ipynb	70.23 KB	
2017‑05‑05
Pruebas antes de GPU, falta un resultado por documentar
TestPAN CLEF corpus.ipynb
# Integración de los Corpus

Para tokenizar el texto existen diferentes metodos. a continuacion una lista de los tokenizadores posibles a utilizar. Evaluar resultados de varios tokenizadores en el proceso. Evualuar el llamado a syntaxnet.
- TweetTokenizer:  Tiene manejo de lexico utilizado en mensajes cortos, smiles, hashtags. Revisar parametros. dejando tokens por palabras
- PunkSentTokenizer: se puede entrenar, dejando oraciones o sentencias  ( http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.punkt ) 
- RegExp ( http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.regexp )
- Simple Tokenizer: utilizan split, division por caracter particular.
- stanford: ?
- Texttiling: division del documento en topicos. interesante para el tema de las etapas de la conversacion. ( http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.texttiling )
- PennTreebank: basado en expresiones regulares.
- El DEfault de la libreria utiliza treebank y punk.

## TODO:
- Importante hacer decode("utf8") antes de tokenizar
- Sacar cada linea de mesaje (tokenizar por mensajes)
- Dividir mensajes y usuario
- Crear lista de mensajes por usurario
- Mesclar las listas
- Tokenizar por otros metodos
- Syntax net de las sentencias
- Reordenar tokens de sentencias, en todas sus combinaciones para detectar los topicos.
- dividir por ":", "-"
- verificar usuarios en el chat
- convertir cada uno en a corpus
- Los chats de PJ quedaron con comentarios entre parentesis, hay que eliminarlos, o extraerlos y ver que se extrae. Con cuidado porque la idea es no borrar puntacion ni informacion.
- Como conectar las caracteristicas de los msjs por la parte de NLP con los conceptos de la red semantica?
- Ver hadoop spark y nltk



- Realizar w2v y revisar 
- alimentar la red cnn con el resultado de w2v
- construir los grupos con todos los datos para la cnn (ajustar el inbalance de datos) y ajustar los hiperparamtros
- Ajustar para que los datos de entrada sean las conversaciones en si.
- construir un modelo con perceptro multicapa con los features.
- a partir de los datos del perceptron y la cnn crear un arbol de desicion.
- hacer consultas a la matriz skipgram


- root@a379e0a83fd0:~/tesis# tensorboard --logdir runs/1487025846/summaries/^C                                                                    



- Ver RNN
- Adicionar mas datos al set de datos.

### Division de grupos de texto

#### Dividir en:

- solo texto de atacantes.
- todo el conjunto de chats.
- grupos de conversaciones por linea en 2 archivos, un archivo de atacantes y otro de otros msjs.
- grupos de conversaciones por linea en 2 archivos, un archivo de atacantes con victimas y otro de otros msjs.


- Los anteriores deben tener un estudio de longitud para ajustar la NN.

### word to vector:

Se puede utilizar, pero hay que evaluar:


from tensorflow.models.embedding import word2vec as w2v

- Hacer graficas de longitud por cantidad de palabras por mensaje y por conversacion.

- Para aumentar la precision se deben dividir las longitudes de conversaciones, y de cada mensaje para los batch, de esa forma la presicion deberia aumentar ya que el contexto se mantendria.

In [0]:
import os
import tensorflow as tf
import re
import nltk
import collections
import numpy as np
import random
import math
import time

from tensorflow.contrib.tensorboard.plugins import projector

import multiprocessing

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10.0, 7.0)

#### Nota:
- Al realizar una revision de los tokens mas frecuentes y menos frecuentes se encintra que hay algunos tokens conformados por emoticones, se va a utilizar un tokenizador casual de NLTK el cual esta fabricado para textos tipo Tweet.

In [0]:
string="b8810fee2f4a71f849f3f7409546d1d9 : see also http://dev.opera.com/articles/view/evolving-the-internet-on-your-phone-des-1/ which does"

string=re.sub(r"\S{15,}\:\s","",string)
string=re.sub(r"\S{15,}\s\:\s","",string)
string=re.sub(r"\S{15,}\,\s","",string)
string=re.sub(r"\S{15,}\s\,\s","",string)
string=re.sub(r"\S{15,}\-\s","",string)
string=re.sub(r"\S{15,}\s\-\s","",string)

print(string)

# : , -

In [0]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\.", " \. ", string)
    #string = re.sub(r"\(", " \( ", string)
    #string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " ? ", string)
    #string = re.sub(r"\s{2,}", " ", string)
    
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)

    #print(type(string))
    #string=string.split()
    return string.strip().lower()

def clean_chats(string):
    string=re.sub(r"\S{15,}\:\s","",string)
    string=re.sub(r"\S{15,}\s\:\s","",string)
    string=re.sub(r"\S{15,}\,\s","",string)
    string=re.sub(r"\S{15,}\s\,\s","",string)
    string=re.sub(r"\S{15,}\-\s","",string)
    string=re.sub(r"\S{15,}\s\-\s","",string)
    return string
def dummy(string):
    string=clean_chats(string)
    return string

In [0]:
%%time
longLines=[]
lines = open('PAN12/full-otros-y-ataques.txt').readlines()
longLines=[w for w in lines if len(w)>500]
lines=[w for w in lines if len(w)<500]
print(len(longLines))


In [0]:
print(longLines[:2])

In [0]:
%%time
numthreads = 12
numlines = 1000
result_list=[]

pool = multiprocessing.Pool(processes=numthreads)
result_list = pool.map_async(dummy, lines, numlines)
print(len(result_list.get()))


In [0]:
f=open("delete.txt","w")
for item in result_list.get():
    for i in item:
        f.write(i)
f.close()


In [0]:
!wc -l PAN12/full-otros-y-ataques.txt
!diff delete.txt PAN12/full-otros-y-ataques.txt | wc -l

In [0]:
#def read_data(f):
#    txt=f.read()
#    txt=clean_chats(txt)
#    print("\nLen at read %s \n" % len(txt))
#    txtaux=clean_str(txt)
#    data = tf.compat.as_str(txtaux).split()
#    return data,txt

####################
############
### Definicion de archivo  
############
####################

#f=open("PAN12/full-otros-y-ataques.txt","r")
f=open("delete.txt","r")
#wordsSimple,txt = read_data(f)
txt=f.read()
#txt=clean_chats(txt)
f.close()

#print('\nData size simple: %s \n' % len(wordsSimple))

#wordsSimpleSet=set(wordsSimple)
#print("\nVocabulary %s \n" % len(wordsSimpleSet))
#vocabulary_size =len(wordsSimpleSet)


#fd=nltk.FreqDist(wordsSimple)

#fd.plot(50, cumulative=False)

#hap=fd.hapaxes()
#print("\nNum words less common %s \n" % len(hap))
#print(hap[:50])


print("\n\nLen at read %s \n" % len(txt))

twtTks=nltk.tokenize.TweetTokenizer(preserve_case=False,reduce_len=True)
txtTks=twtTks.tokenize(txt)
print('\nData size Tweet Tokenizer %s \n' % len(txtTks))

txtTksSet=set(txtTks)
print("\nVocabulary %s \n" % len(txtTksSet))
vocabulary_size =len(txtTksSet)

txtTksFd=nltk.FreqDist(txtTks)

txtTksFd.plot(50, cumulative=False)

txtTksFdHap=txtTksFd.hapaxes()
print("\nNum words less common %s \n" % len(txtTksFdHap))
print(txtTksFdHap[:50])

print("# DONE")

### OJO:
- Hay que ensayar con los 2 tipo de tokenizadores
- y se ESTAN eliminando los Hapaxes

In [0]:
#vocabulary_size = len(wordsSet)
#words=wordsSimple
#vocabulary_size = len(txtTksSet)
#words=txtTks
#vocabulary_size = len(txtTksSet)-len(txtTksFdHap)+1
words=txtTks

tmpset=set(txtTksFdHap)
print("data set: ",len(words))                
modwords=[w if w not in tmpset else "UNKVERSION2" for w in words]
print("con las palabras menos utilizadas modificadas:", len(modwords))
longWords=[w for w in modwords if len(w)>20]

print("Cantidad de palabras extensas:",len(longWords))
longWordsSet=set(longWords)
print("Diccionario de palabras extensas:",len(longWordsSet))

modwFd=nltk.FreqDist(modwords)
modwFd.plot(50, cumulative=False)

longWordsFd=nltk.FreqDist(longWords)
longWordsFd.plot(50, cumulative=False)

print(longWordsFd.hapaxes())

shortWords=[w for w in modwords if len(w)<20]

words=shortWords
vocabulary_size=len(set(words))
print("Vocab size",vocabulary_size)


In [0]:
nltktxt=nltk.Text(txtTks)
mc=longWordsFd.most_common(10)
mc2=[x[0] for x in mc]
nltktxt.dispersion_plot(mc2)

In [0]:
nltktxt.concordance(mc2[0])

In [0]:
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary
    
    
data, count, dictionary, reverse_dictionary = build_dataset(words)
#del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])
data_index = 0


- Explicacion de los parametros:   http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/

In [0]:
data_index=0
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=16, num_skips=2, skip_window=1)# 16 a 512, 8,2,1
print("batch: ",len(batch))
print("labels: ",len(labels))
for i in range(16):
    print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])


def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [0]:
#!rm -r embed

In [0]:
%%time
batch_size = 128  # 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size],name="train_input")
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1],name="train_labels")
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32,name="valid_dataset")

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0),name="embeddings")
        #variable_summaries(embeddings)
        embed = tf.nn.embedding_lookup(embeddings, train_inputs,name="lookup")

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)),name="nce_Ws")
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]),name="nce_Bs")

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                    b num_classes=vocabulary_size))
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)
    
    # For embedd visualization
    #embedding_var = tf.Variable(normalized_embeddings[:500],name="embed_to_save")

    
    # Add variable initializer.
    init = tf.global_variables_initializer()

In [0]:
%%time
num_steps = 10001
embed_viz_size = 10000
data_index=0
timestamp = str(int(time.time()))

print("Lanzamiento No: ", timestamp)
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter("embed/" + timestamp + '/train',
                                          session.graph)
    test_writer = tf.summary.FileWriter("embed/" + timestamp + '/test')
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        summary,_, loss_val = session.run([merged,optimizer, loss], feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if (step % 10000 == 0) and (False):
            #saver = tf.train.Saver()
            #saver.save(session, os.path.join("embed", "model.ckpt"), step)
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()
    
    # For embedd visualization
    print("\n#### Skip-gram end")
    # Link this tensor to its metadata file (e.g. labels).
    tsvLabels = [re.sub(r"\n", "",reverse_dictionary[i]).encode('utf8')+"\n" for i in xrange(embed_viz_size)]
#    tsvLabels = [re.sub(r"\n", "",reverse_dictionary[i])+"\n" for i in xrange(embed_viz_size)]
    fLabels=open("embed/"+timestamp+"/train/metadata.tsv","w")
    print("tsvLabels: ")
    print(len(tsvLabels))
    print(type(tsvLabels))
    #print(tsvLabels)
    fLabels.writelines(tsvLabels)
    fLabels.close()
    
    #embedding_var = tf.Variable(normalized_embeddings[:500],name="embed_to_save")
    #embedding_var = tf.Variable(final_embeddings[:500],name="embed_to_save")
    embedding_var = tf.Variable(normalized_embeddings[:embed_viz_size], name='embedding_labels')
    session.run(embedding_var.initializer)
    embeddingTot = tf.Variable(normalized_embeddings, name='embeddingsTot')
    session.run(embeddingTot.initializer)
    # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
    config = projector.ProjectorConfig()
    # Use the same LOG_DIR where you stored your checkpoint.
    summary_writer = tf.summary.FileWriter("embed/"+timestamp+"/train")
    # You can add multiple embeddings. Here we add only one.
    embeddingConf = config.embeddings.add()
    embeddingConf.tensor_name = embedding_var.name
    embeddingConf.metadata_path = os.path.join("embed/"+timestamp+"/train/", 'metadata.tsv')
    # Saves a configuration file that TensorBoard will read during startup.
    projector.visualize_embeddings(summary_writer, config)
    # save checkpoint
    saver = tf.train.Saver([embedding_var,embeddingTot])
    saver.save(session, os.path.join("embed/"+timestamp+"/train/", "modelskip.ckpt"), 1)
    print("\n#### embedds")
    
    
print("index: ",data_index)
print("data len: ",len(data))

# Integracion de embeds con Lexicon, diccionario Empath (U. Stanford) 

In [0]:
import os
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib.tensorboard.plugins import projector

import re
import nltk
from nltk.tokenize import TweetTokenizer

import collections
import numpy as np
import random
import math
import time

import multiprocessing
import itertools


%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 7.0)

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

In [0]:
%%time

NLTK_TWEET_TOKENIZER = TweetTokenizer(preserve_case=False,reduce_len=True)

def tokenizerFn(iterator):
    for value in iterator:
        try:
            yield NLTK_TWEET_TOKENIZER.tokenize(value)
        except TypeError:       # this is a hack to avoid the error
            yield []     

def clean_chats(string):
    string=re.sub(r"\S{15,}\:\s","",string)
    string=re.sub(r"\S{15,}\s\:\s","",string)
    string=re.sub(r"\S{15,}\,\s","",string)
    string=re.sub(r"\S{15,}\s\,\s","",string)
    string=re.sub(r"\S{15,}\-\s","",string)
    string=re.sub(r"\S{15,}\s\-\s","",string)
    string=re.sub(r"\S{20,}","",string)
    return string.strip()
def dummy(string):
    string=clean_chats(string)
    return string


#lines = open('PAN12/conversaciones-ataques.txt').readlines()
#lines = open('PAN12/atacantes.txt').readlines()
lines = open('PAN12/full-otros-y-ataques.txt').readlines()
lines=[unicode(w,errors='ignore') for w in lines if len(w)<500]

numthreads = 12
numlines = 1000
result_list=[]

pool = multiprocessing.Pool(processes=numthreads)
result_list = pool.map_async(dummy, lines, numlines)

txt=result_list.get()
#txt=" ".join(result_list.get())
#read con decode
#para verlo bien .encode("unicode_escape")
# del var

#############
#############

txtLen=50

############
############

vocab_processor = learn.preprocessing.VocabularyProcessor(txtLen,min_frequency=1,tokenizer_fn=tokenizerFn)

txtTransform=list(vocab_processor.fit_transform(txt))

print("Vocabulary:  %s" % len(vocab_processor.vocabulary_))
print("Messages: %s" % len(txtTransform))

dataNp=np.array(txtTransform)
data=dataNp.flat
print("Data len: %s" % len(data))

##### Eliminar tokens Repetidos en el FLAT
data=[k for k, g in itertools.groupby(data)]
print("Total Tokens: %s" % len(data))

# Divide los mensajes largos en N numeros de lineas y elimina las lineas que sean ceros.

dataNp=np.array(txtTransform)
print("Messages full: %s" % len(dataNp))

x=dataNp
b=[]
for i in xrange(len(x)):
    a=np.split(x[i],5,0) # <<<<-------- Dividir txtLen en cuantas lineas??
    for aa in a: 
        b.append(aa)
x=np.array(b)
del(b)
print("Messages post split: %s" % len(x))

delRow=np.nonzero(x.sum(axis=1) == 0)
x = np.delete(x, delRow, axis=0)

print("Messages post delete zeros: %s" % len(x))


dataNp=x
del(x)

data=dataNp.flat

print(dataNp[:10])
print("\n")
print(data[:100])
print("Total Tokens: %s" % len(data))

In [0]:
data_index=0
i_message=0

hist, bins = np.histogram(data,range=(0,50), bins=50,density=False)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()
print([float(hi)/len(data) for hi in hist[:10]])


In [0]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global i_message
    global txtLen
    global messages_batch
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    i_message+=1
    ##aux_data_index=(txtLen*messages_batch*i_message)-(skip_window*2)
    aux_data_index=(txtLen*i_message)-(skip_window*2)
    if((aux_data_index>len(data)) or ((aux_data_index+batch_size)>len(data))):
        data_index = (data_index + len(data) - span) % len(data)
    else:
        data_index=aux_data_index
    return batch, labels

#messages_batch=1

num_skips=2
skip_window=1
#batch_size=100
#batch_size=(messages_batch*txtLen*num_skips)-(num_skips*skip_window*2)
#batch_size=(messages_batch*txtLen*num_skips)-(skip_window*2)
#batch_size=(messages_batch*txtLen*num_skips)+(skip_window)
#batch_size=(messages_batch*txtLen*num_skips)
batch_size=(txtLen*num_skips)
batch, labels = generate_batch(batch_size, num_skips, skip_window)# 16 a 512, 8,2,1

print("batch: ",len(batch))
print("labels: ",len(labels))
print("data-index",data_index)
#for i in range(batch_size):
#    print(i,batch[i], vocab_processor.vocabulary_.reverse(batch[i]),
#        '->', labels[i, 0], vocab_processor.vocabulary_.reverse(labels[i, 0]))


In [0]:
%%time

vocabulary_size=len(vocab_processor.vocabulary_)
print("Vocabulary: %s " % vocabulary_size)
#messages_batch=1
embedding_size = 128  # Dim embedding vector.
skip_window = 2       # How many words left and right. 
num_skips = 2         # How many times to reuse an input to generate a label. Para dar un poco de euristica
#batch_size = 128  # 128
#batch_size=(messages_batch*txtLen*num_skips)
batch_size=(txtLen*num_skips)
print("Batch size: %s" % batch_size)

# Random validation set to sample nearest neighbors.
# limit the validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 10     # Random set of words to evaluate similarity on.
valid_window = 500  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.


graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size],name="train_input")
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1],name="train_labels")
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32,name="valid_dataset")

    with tf.device('/cpu:0'),tf.name_scope("embeds"):

        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0),name="embeddings")
        #variable_summaries(embeddings)
        embed = tf.nn.embedding_lookup(embeddings, train_inputs,name="lookup")

    with tf.device('/cpu:0'),tf.name_scope("nce-loss"):
        # NCE loss, noise-contrastive estimation loss , a logistic regression model
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                                      stddev=1.0 / math.sqrt(embedding_size)),
                                  name="nce_Ws")
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]),name="nce_Bs")

        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=num_sampled,
                                             num_classes=vocabulary_size))
    tf.summary.scalar('loss', loss)
    
    #optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

    with tf.device('/cpu:0'),tf.name_scope("cosine-sim"):
        # cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    
    # Add variable initializer.
    init = tf.global_variables_initializer()

In [0]:
%%time
num_steps = 150001 # 1M
embed_viz_size = 2000 #10K
data_index=0
i_message=0
timestamp = str(int(time.time()))

print("Lanzamiento No: ", timestamp)

with tf.Session(graph=graph) as session:

    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter("embed/" + timestamp + '/train',
                                          session.graph)
    init.run()
    
    if(1):   # <<<<<<<<<<<<<<--------------------- Restore previous embedds
        saverR = tf.train.Saver({"embeddingsTot": embeddings})
        saverR.restore(session, "embed/"+"1491808882"+"/train/modelskip.ckpt-1")
        print("EMBEDS RESTORED  <<<<<<<<<<<<")
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        summary,_, loss_val = session.run([merged,optimizer, loss], feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        average_loss += loss_val

        if step % 1500 == 0:    # <<<<<<<-------------calculate average loss
            if step > 0:
                average_loss /= 1500
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        if (step % 15000 == 0): # 100K   <<<<<<<<<----------- Similarity
#        if (step % 10000 == 0) and (False):
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = vocab_processor.vocabulary_.reverse(valid_examples[i])
                top_k = 5  #  <<<<---------------    number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = vocab_processor.vocabulary_.reverse(nearest[k])
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()
    
    # For embedd visualization
    print("\n#### Skip-gram end")
    tsvLabels = [re.sub(r"\n", "",vocab_processor.vocabulary_.reverse(i)).encode('utf8')+"\n" for i in xrange(embed_viz_size)]
    fLabels=open("embed/"+timestamp+"/train/metadata.tsv","w")
    print("tsvLabels: ")
    print(len(tsvLabels))
    print(type(tsvLabels))
    fLabels.writelines(tsvLabels)
    fLabels.close()
    
    embedding_var = tf.Variable(normalized_embeddings[:embed_viz_size], name='embedding_labels')
    session.run(embedding_var.initializer)
    embeddingTotNorm = tf.Variable(normalized_embeddings, name='embeddingsTotNorm')
    session.run(embeddingTotNorm.initializer)
    embeddingTot = tf.Variable(embeddings, name='embeddingsTot')
    session.run(embeddingTot.initializer)
    
    # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
    config = projector.ProjectorConfig()
    # Use the same LOG_DIR where you stored your checkpoint.
    summary_writer = tf.summary.FileWriter("embed/"+timestamp+"/train")
    # You can add multiple embeddings. Here we add only one.
    embeddingConf = config.embeddings.add()
    embeddingConf.tensor_name = embedding_var.name
    embeddingConf.metadata_path = os.path.join("embed/"+timestamp+"/train/", 'metadata.tsv')
    # Saves a configuration file that TensorBoard will read during startup.
    projector.visualize_embeddings(summary_writer, config)
    # save checkpoint
    saver = tf.train.Saver([embedding_var,embeddingTotNorm,embeddingTot])
    saver.save(session, os.path.join("embed/"+timestamp+"/train/", "modelskip.ckpt"), 1)
    
    # Vocabulary
    vocab_processor.save("embed/"+timestamp+"/train/vocabulary.txt")
    print("\n#### embedds saved")
    
    
print("index: ",data_index)
print("data len: ",len(data))

In [0]:
#!rm -r embed

# Recuperar embeds

In [0]:
import tensorflow as tf
from tensorflow.contrib import learn
import numpy as np
import nltk
from nltk.tokenize import TweetTokenizer


modelN="1490055956"
timestamp=modelN
vocabulary_size=2757
embedding_size=128
embeddingsTotNorm = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0),name="embeddingsTotNorm")
saver = tf.train.Saver({"embeddingsTotNorm": embeddingsTotNorm})
#saver = tf.train.Saver()

with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "embed/"+timestamp+"/train/modelskip.ckpt-1")
    embedNp=sess.run(embeddingsTotNorm)
    print("Model restored.")
    print("Embeds: %s" % len(embedNp))

In [0]:
NLTK_TWEET_TOKENIZER = TweetTokenizer(preserve_case=False,reduce_len=True)

def tokenizerFn(iterator):
    for value in iterator:
        try:
            yield NLTK_TWEET_TOKENIZER.tokenize(value)
        except TypeError:       # this is a hack to avoid the error
            yield []     

txtLen=50

vocab_processor = learn.preprocessing.VocabularyProcessor(txtLen,min_frequency=1,tokenizer_fn=tokenizerFn)
vr=vocab_processor.restore("embed/"+timestamp+"/train/vocabulary.txt")
len(vr.vocabulary_)

In [0]:
print(vr.vocabulary_.reverse(0))

# Empath

In [0]:
from empath import Empath
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (15.0, 10.0)


lexicon = Empath()
print(len(txt))

In [0]:
NLTK_TWEET_TOKENIZER_EMPATH = TweetTokenizer(preserve_case=False,reduce_len=True)

def tokenizerFnDoc(doc):
    for tk in NLTK_TWEET_TOKENIZER_EMPATH.tokenize(doc):
        try:
            yield tk
        except TypeError:       # this is a hack to avoid the error
            yield []  

def clean_chats(string):
    string=re.sub(r"\S{15,}\:\s","",string)
    string=re.sub(r"\S{15,}\s\:\s","",string)
    string=re.sub(r"\S{15,}\,\s","",string)
    string=re.sub(r"\S{15,}\s\,\s","",string)
    string=re.sub(r"\S{15,}\-\s","",string)
    string=re.sub(r"\S{15,}\s\-\s","",string)
    string=re.sub(r"\S{20,}","",string)
    return string.strip()
def dummy(string):
    string=clean_chats(string)
    return string


lines = open('PAN12/atacantes.txt').readlines()
lines=[unicode(w,errors='ignore') for w in lines if len(w)<500]

numthreads = 12
numlines = 1000
result_list=[]

pool = multiprocessing.Pool(processes=numthreads)
result_list = pool.map_async(dummy, lines, numlines)

txt=result_list.get()

In [0]:
#lex=lexicon.analyze(txt,tokenizer=tokenizerFnDoc,normalize=True)
lex=lexicon.analyze(txt,tokenizer=tokenizerFnDoc,normalize=False)
#lex=lexicon.analyze(txt,normalize=False)


In [0]:
import operator

########
########
##  ORDEN POR VALOR

lexLstSort =sorted(lex.items(), key=operator.itemgetter(1))

lexLstSort=list(reversed(lexLstSort))
lexv=[a[1] for a in lexLstSort[:30] if (a[1]>0)]
lexl=[a[0] for a in lexLstSort[:30] if (a[1]>0)]

print(lexl[:5])
print(lexv[:5])

alphab = lexl
frequencies = lexv

pos = np.arange(len(alphab))
width = 1.0     # gives histogram aspect to the bar diagram
ax = plt.axes()
ax.set_xticks(pos + (width / 2))
ax.set_xticklabels(alphab)
plt.xticks( rotation='vertical')
plt.bar(pos, frequencies, width)
plt.show()

######
######
##   ORDEN EMPATH

lexv=[a[1] for a in lex.items() if (a[1]>0)]
lexl=[a[0] for a in lex.items() if (a[1]>0)]

alphab = lexl
frequencies = lexv

pos = np.arange(len(alphab))
width = 1.0     # gives histogram aspect to the bar diagram
ax = plt.axes()
ax.set_xticks(pos + (width / 2))
ax.set_xticklabels(alphab)
plt.xticks( rotation='vertical')
plt.bar(pos, frequencies, width)
plt.show()


In [0]:
from sklearn.metrics.pairwise import pairwise_distances
#print(embedNp[:2])
res=pairwise_distances(embedNp, metric='cosine',n_jobs=-2)
print(len(res))


In [0]:
lexCats=lexicon.cats
#lexCatsLst=[item[0] for item in lexicon.cats.items()]
#lexItemsLst=[item[1] for item in lexicon.cats.items()]


vocListNp=np.array([vr.vocabulary_.reverse(i) for i in xrange(len(vr.vocabulary_))])
print("voc: %s" % len(vocList))
network={}
for cat in lexLstSort[:10]:
    itemCatNp=np.array(lexCats[cat[0]])
    intersec=np.intersect1d(vocListNp,itemCatNp)
    b=[]
    for term in intersec:
        i=vr.vocabulary_.get(term)
        row=res[i]
        row[i]=1
        ii=np.argmin(row)
        b.append(vr.vocabulary_.reverse(ii))
    intersecAux=list(intersec)+b
    intersecAux=list(set(intersecAux))
    network[cat[0]]=intersecAux
len(network)

In [0]:
class User(object):
    def __init__(self, s, t):
        self.source = s
        self.target = t
networkLst=[]
nodes=[]
for a in network:
    nodes.append(a)
    for ai in network[a]:
        networkLst.append(User(a,ai))
        nodes.append(ai)
#print(nodes)

#nodes=list(set(nodes))


import json

def jdefault(o):
    return o.__dict__


In [0]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [0]:
HTML('''
<script src="lib/sigmajs/sigma.min.js"></script>
<script src="js/sigma-add-method-neighbors.js"></script>
''')

In [0]:
random.seed(42)

graph_data = { 'nodes': [], 'edges': [] }

nodesU=[]
for i in nodes:
    if(type(i)==np.unicode_):
        dd=i.encode('utf8')
        dd=str(dd)
    else:
        dd=str(i)
    nodesU.append(dd)
    
print(len(nodesU))
nodesU=list(set(nodesU))
print(len(nodesU))
print(len(networkLst))


for i in nodesU:
    
    siz=lex.get(i)
    if siz is None:
        siz=0.3
    else:
        siz=siz/750 #1000
    graph_data['nodes'].append({
            "id": str(i),
            "label": str(i),
            "x": random.uniform(0,1),
            "y": random.uniform(0,1),
            "size": siz
        })

ii=0
for j in networkLst:
    x_center = random.uniform(0,1)
    y_center = random.uniform(0,1)
    x_dist = random.uniform(0.1,0.5)
    y_dist = random.uniform(0.2,0.5)
    graph_data['edges'].append({
            "id": "e" + str(ii),
            "source": str(j.source),
            "target": str(j.target)
        })
    ii+=1

In [0]:
pd.DataFrame(graph_data['nodes']).head(1)


In [0]:
pd.DataFrame(graph_data['edges']).head(1)


In [0]:
js_text_template = Template(open('js/sigma-graph.js','r').read())

js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div'})

html_template = Template('''
<div id="graph-div" style="width: 900px;max-width: 900px;height:800px"></div>
<script> $js_text </script>
''')
HTML(html_template.substitute({'js_text': js_text}))